# Networks, Clients, and Servers

Clients and servers are applications or processes that can run locally on a single computer or remotely across a network of computers. 

As explained in the following sections,the resources required for this type of application are IP addresses, sockets,and threads.

## 1 IP Addresses

Every computer on a network has a unique identifier called an **IP address** 

* `IP stands for Internet Protocol`.

This address can be specified either as an **IP number** or as an **IP name**. 

An IP number typically has the form 
```text
ddd.ddd.ddd.ddd, 
```
where each $d$ is a digit. The number of digits to the right or the left of a decimal point may vary but does
not exceed `three`. 

For example, the IP number of the author’s office computer might be `137.112.194.77`. Because IP numbers can be difficult to remember, people customarily use an `IP name` to specify an IP address. For example, the IP name of the author’s computer might be `lambertk`.

Python’s [socket](https://docs.python.org/3/library/socket.html) module includes two functions that can look up these items of information.
These functions are listed in Table, followed by a short session showing their use.

|socket Function|What It Does|
|:-------:|:-----------:|
|gethostname()| Returns the IP name of the host computer running the Python interpreter.<br>Raises an exception if the computer does not have an IP address.|
|gethostbyname(ipName)| Returns the IP number of the computer whose IP name is ipName. <br/>RRaises an exception if ipName cannot be found.|


In [ ]:
from socket import *
hostname=gethostname()
print(hostname+" address is "+gethostbyname(hostname))
hosturl="seu.edu.cn"
print(hosturl+" address is :"+gethostbyname(hosturl))


### The local host  

When developing a network application, the programmer can first try it out on **a local host**—that is, on 

* `a standalone computer that may or may not be connected to the Internet`.

The computer’s IP name in this case is **"localhost"**, a name that is standard for any computer. 

The IP number of a computer that acts as **a local host** is distinct from its IP number as **an Internet host**, as shown in the next session:


In [ ]:
from socket import *
print(gethostbyname(gethostname()))
print(gethostbyname("localhost"))

When the programmer is satisfied that the application is `working correctly on a local host`, the application can then be deployed on the Internet host simply by `changing` the IP address.

In the discussion that follows, we use `a local host` to develop network applications.

## 2 Ports, Sockets, Servers, and Clients

### 2.1 ports

Clients connect to servers via objects known as **ports**. 

A `port` serves as a `channel` through which several clients can exchange data with the same server or with different servers.

Ports are usually specified by `numbers`. 

Some ports are dedicated to `special servers or tasks1. 

For example, 

* Port number $80$ is reserved for a **Web server**, and so forth.

Most computers also have hundreds or even thousands of **free ports** available for use by any network  applications.



### 2.2 Sockets  and  server-client

#### 2.2.1 Sockets 

We write a CPU server script in Python to handle requests from many clients.

The client connects to the server, and the two programs engage in a continuous communication until one of them, usually the client, decides to quit.

To do this, you need to use **a socket**. 

* A `socket` is an object that serves as a `communication` link between a single server process and a single client process. 
*  套接字(Socket)：进程之间通信端点的抽象

You can create and open **several** sockets on the same port of a host computer. 

* the relationships between `a host computer,ports, servers, clients, and sockets`.


![host-port-socket](./img/linux/host-port-socket.jpg)


#### 2.2.2 a socket communication

* create a socket object

* bind() 绑定

* listen() 侦听

* accept()  # the loop  to accecp the client

  * recv()  # the nested loop to communicate with the clent

  * send()  # the nested loopto communicate with the clent

* close() # close the connection of the clien

![socket_server_client](./img/linux/socket_server_client.jpg)


**Server-client Output** 


![echo-server-client](./img/linux/echo-server-client.jpg)

To see the output,

* **first** run the socket **server** program. 

* Then run the socket **client** program.

After that,client send message to server 

Then  server receive the message and send CPU date to client. 

At last, press `Ctrl+C` ，send **"bye"** to the server to close the sockets and terminate both program. .

In the `first` shot， the CPU server script
  * the CPU server script is `launched` in a terminal window, and
  * it’s `waiting` for a `connection`. 

In the `second` shot, 
  
 * one `client` is launched in a separate terminal window 
 
 * It has connected to the server and received the CPU data 
 
The `third` shot shows

  * the updates to the server’s window after it has served the client. 




### 2.3  Sever

#### .2.3.1 sever script

In [5]:
%%file ./code/python/echo-server-cpu.py
from socket import *

import psutil
from codecs import encode

def get_data():
    return psutil.cpu_percent()

def server_program():
    bufsize = 1024
    host = "localhost"
    #host = gethostname()
    port = 5000  # initiate port no above 1024
    
    server_socket = socket(AF_INET, SOCK_STREAM)# get instance
    # server_socket = socket()
    # look closely. The bind() function takes tuple as argument
    address = (host, port)
    server_socket.bind(address)  # bind host address and port together

    # configure how many client the server can listen simultaneously
    server_socket.listen(5)
    print("Waiting for connection ...")
    
    # accept new connection
    while True:
        conn, address = server_socket.accept()  
        print("Connection from: " + str(address))
        # the nested loop
        while True:
            try:
                # receive data stream. it won't accept data packet greater than 1024 bytes
                data = conn.recv(bufsize).decode()
                if not data:
                    # if data is not received break
                    break
                print("from connected user: " + str(data))
                data = encode(str(round(get_data(),4)),"ascii")
                conn.send(data)  # send data to the client
            except:
                break
            
        conn.close()  # close the connection
    
if __name__ == '__main__':
    server_program()


Writing ./code/python/echo-server-cpu.py


#### 2.3.2 Code interpretation

A `socket` resembles a `file` object, in that the programmer opens it, receives data from it, and closes it when finished.

We now explain these steps in our server script in more detail.


* create a socket object

* bind() 绑定

* listen() 侦听

* accept()  # the loop  to accecp the client

  * recv()  # the nested loop to communicate with the clent

  * send()  # the nested loopto communicate with the clent

* close() # close the connection of the clien



**1 create a socket object** 

The server script uses the `IP address` and `port number`information to create a socket object 

```python
host = "localhost"
port = 5000  # initiate port no above 1024
server_socket = socket(AF_INET, SOCK_STREAM)# get instance 
```

The script creates a socket by running the function **socket()** in the socket module. 

This function returns a new socket object, when given

* `a socket family` 

* `a socket type` 

as arguments.We use the family **AF_INET** and the type **SOCK_STREAM**, 

* **SOCK_STREAM**: the default protocol that’s used is the `Transmission Control Protocol (TCP)`. 

**2 listen 侦听**

**First**  the socket is bound to this address by running its **bind** method.
```python
address = (host, port)
server.bind(address)
```
**Second,**  the socket then is made to listen for up to `five` requests at a time from clients by running its listen method. If you want the server to handle more concurrent requests before rejecting additional ones, you can increase this number.
```python
server.listen(5)
```

**3 accept()**

```python
# accept new connection
    conn, address = server_socket.accept()  
    print("Connection from: " + str(address))
```

**4 main loop and the nested loop**


The nested loop for the connected client
```python
while True:
        # receive data stream. it won't accept data packet greater than 1024 bytes
        data = conn.recv(bufsize).decode()
        if not data:
            # if data is not received break
            break
        print("from connected user: " + str(data))
        data = encode(str(round(get_data(),4)),"ascii")
        conn.send(data)  # send data to the client


```

the server then enters `a nested  loop`. 

This loop engages the server in a `continuous` conversation with the client. 

The server `receives` a message from the client. 

```python
# receive data stream. it won't accept data packet greater than 1024 bytes
        data = conn.recv(bufsize).decode()
        if not data:
            # if data is not received break
            break
        print("from connected user: " + str(data))  
            
```
Our script binds the variables client and address to these values and uses them in the next steps.


The script prints the client’s address, and then sends the current CPU to the client by
running the send method with the client’s socket.

```python
 data = encode(str(round(get_data(),4)),"ascii")
        conn.send(data)  # send data to the client

```

The send method expects `a bytes object`
as an argument. You create a bytes object from a string by calling the built-in bytes function,


**5  close the connection**
```python
conn.close()  # close the connection
```


### 2.4 Client


#### 2.4.1 client script


In [4]:
%%file ./code/python/echo-client-cpu.py

from socket import *
import time
from codecs import decode


def client_program():
    # host = socket.gethostname()
    host = "localhost"  # as both code is running on same pc
    port = 5000  # socket server port number

    client_socket = socket(AF_INET, SOCK_STREAM)  # get instance
    # client_socket = socket.socket()  # instantiate
    client_socket.connect((host, port))  # connect to the server

    message = "cpu"
    while True:
        try:
            client_socket.send(message.encode())  # send message
            data = decode(client_socket.recv(1024), "ascii")
            print('Received from server: ' + data)  # show in terminal
            time.sleep(1)
            message = "cpu"
        except KeyboardInterrupt:  # Press 'Ctrl + C' to  close the connection
            ans = input('\nDo you want to continue(y/n) :')
            if ans == 'y':
                continue
            else:
                break
    
    client_socket.close()  # close the connection


if __name__ == '__main__':
    client_program()


Overwriting ./code/python/echo-client-cpu.py


#### 2.4.2  Code interpretation

1. connect

2. main loop
  
  * send(message.encode()) 
  * client_socket.recv(1024)
  
3. client_socket.close()  

**1.socket**
```python
 # host = socket.gethostname() 
    host = "localhost"# as both code is running on same pc
    port = 5000  # socket server port number
    
    client_socket = socket(AF_INET, SOCK_STREAM)# get instance
```   
**connect()**
```python
ADDRESS = (HOST, PORT)
server.connect(ADDRESS)
```
To connect the socket to a host computer, one runs the socket’s **connect** method. 

This method expects as an argument a tuple containing

* `the host’s IP address and a port number`.

In this case, these values are `"localhost" and 5000`, respectively. These two values should be the same as the ones used in the server script.


**3 mainloop: send() and recv()** 

**send()** 

**recv()**

```python
cpu_percente = decode(server.recv(BUFSIZE), "ascii")
```
To obtain information sent by the server, the client script runs the socket’s **recv()** method.

This method expects as an argument the `maximum` size in **bytes** of the data to be read from the socket.

The **recv()** method returns an object of type `bytes` 

You convert this to a string by calling the codecs function **decode()**, with the encoding **"ascii"** as the second argument.

**close()**
```pythom
server.close()
```
After the client script has printed the string read from the socket, the script closes the connection to the server by running the socket’s **close** method.


## 3 Threading Multi-Connection Server


The **server** definitely has its `limitations`. 

* It serves **only one** client and then exits 

![socket-single-client](./img/linux/socket-single-client.png)



### 3.1 Multi-Connection Server

we’ll create a server that handles **multiple** connections using **threading** module.

**Thread method**

In [2]:
%%file ./code/python/multiconn-server.py

import psutil
from socket import *
import threading
from codecs import encode

class ClientThread(threading.Thread):

    def __init__(self,conn,address, bufsize):
        threading.Thread.__init__(self)
        self.bufsize=bufsize
        self.conn = conn
        self.address=address
        print ("New connection added: ", address)
    
    def run(self):
        # the nested loop
        while True:
            try:
                # receive data stream. it won't accept data packet greater than 1024 bytes
                data = self.conn.recv(self.bufsize).decode()
                if not data:
                    # if data is not received break
                    break
                print("from connected user: " + str(data))
                data = encode(str(round(get_data(),4)),"ascii")
                self.conn.send(data)  # send data to the client
            except:
                break
   
        self.conn.close()# close the connection
        print ("Client at ", self.address , " disconnected...")

        
def get_data():
    return psutil.cpu_percent()

def server_program():
    bufsize = 1024
    host = "localhost"
    #host = gethostname()
    port = 5000  # initiate port no above 1024
    
    server_socket = socket(AF_INET, SOCK_STREAM)# get instance
    # server_socket = socket()
    # look closely. The bind() function takes tuple as argument
    address = (host, port)
    server_socket.bind(address)  # bind host address and port together

    # configure how many client the server can listen simultaneously
    server_socket.listen(5)
    print("Waiting for connection ...")
    
    while True:
        conn, address =  server_socket.accept()
        newthread = ClientThread(conn, address, bufsize)
        newthread.start()
            
if __name__ == '__main__':
    server_program()           


Writing ./code/python/multiconn-server.py


### 3.2 Multi-Connection Client



In [3]:
%%file ./code/python/echo-client-cpu.py

from socket import *
import time
from codecs import decode


def client_program():
    # host = socket.gethostname()
    host = "localhost"  # as both code is running on same pc
    port = 5000  # socket server port number

    client_socket = socket(AF_INET, SOCK_STREAM)  # get instance
    # client_socket = socket.socket()  # instantiate
    client_socket.connect((host, port))  # connect to the server

    message = "cpu"
    while True:
        try:
            client_socket.send(message.encode())  # send message
            data = decode(client_socket.recv(1024), "ascii")
            print('Received from server: ' + data)  # show in terminal
            time.sleep(1)
            message = "cpu"
        except KeyboardInterrupt:  # Press 'Ctrl + C' to  close the connection
            ans = input('\nDo you want to continue(y/n) :')
            if ans == 'y':
                continue
            else:
                break
    
    client_socket.close()  # close the connection


if __name__ == '__main__':
    client_program()


Writing ./code/python/echo-client-cpu.py


![socket-multi-client](./img/linux/socket-multi-client.png)

## Reference

[Python:Networking and Interprocess Communication](https://docs.python.org/3/library/ipc.html)

* [socket](https://docs.python.org/3/library/socket.html)

[[Python:Internet Protocols and Support](https://docs.python.org/3/library/internet.html)

* [socketserver — A framework for network servers](https://docs.python.org/3/library/socketserver.html)
